In [21]:
from diff_match_patch import diff_match_patch

def fuzzy_match(search, replace, file_content):
    dmp = diff_match_patch()
    # Source: https://github.com/Aider-AI/aider/blob/4251e976b3aa52c2a3af08da4b203d4d524c8e92/aider/coders/search_replace.py#L280C1-L293
    dmp.Diff_Timeout = 5
    dmp.Match_Threshold = 0.95
    dmp.Match_Distance = 500
    dmp.Match_MaxBits = 128
    dmp.Patch_Margin = 32

    diff = dmp.diff_main(search, replace, None)
    dmp.diff_cleanupSemantic(diff)
    dmp.diff_cleanupEfficiency(diff)

    patches = dmp.patch_make(search, diff)

    new_content, success = dmp.patch_apply(patches, file_content)
    all_success = success and all(success)
    print("Fuzzy match success: " + str(all_success))
    print(new_content)


In [22]:
file_content = """
class Foo:
    def foo(self, x):
        return x + 1
    
    def bar(self, x):
        if x > 10:
            return x + 2
        else:
            return x + 3
"""
# Test case 1: Simple method signature change
search = "def foo(self, x):"
replace = "def foo(self, x, y=0):"


fuzzy_match(search, replace, file_content)

Fuzzy match success: True

class Foo:
    def foo(self, x, y=0):
        return x + 1
    
    def bar(self, x):
        if x > 10:
            return x + 2
        else:
            return x + 3



In [25]:
# Test case 2: Fuzzy whitespace differences
search = "def foo(self, x  ):"
replace = "def  foo(self,x ):"
fuzzy_match(search, replace, file_content)

Fuzzy match success: True

class Foo:
    def  foo(self,xx):
        return x + 1
    
    def bar(self, x):
        if x > 10:
            return x + 2
        else:
            return x + 3



In [27]:
# Test case 3: Multi-line method change with indentation differences
search = """def bar(self, x):
        if x > 10:
            return x + 2"""
replace = """def bar(self, x):
        if x > 10:
            return x * 2
            # Multiply instead of add"""
fuzzy_match(search, replace, file_content)




Fuzzy match success: True

class Foo:
    def foo(self, x):
        return x + 1
    
    def bar(self, x):
        if x > 10:
            return x + 2
        else:
            return x * 3

            # Multiply instead of add


In [29]:

# Test case 4: Multi-line with extra whitespace and newlines
search = """    def bar(self, x):

        if x > 10:
            return x + 2
        else:
            return x + 3"""
replace = """    def bar(self, x):
        if x > 10:
            return x + 2
        else:
            return x + 3
            # Extra comment"""
fuzzy_match(search, replace, file_content)


Fuzzy match success: True

class Foo:
    def foo(self, x):
        return x + 1
    
    def bar(self, x):
        if x > 10:
            return x + 2
        else:
            return x + 3

            # Extra comment


In [30]:

# Test case 5: Fuzzy matching with similar but not identical code
search = """    def bar(self, x):
        if x > 10:
            return x + 2
        else:
            return x + 3"""
replace = """    def bar(self, x):
        if x >= 10:  # Changed comparison
            return x + 2.0  # Added decimal
        else:
            return x + 3"""
fuzzy_match(search, replace, file_content)

Fuzzy match success: True

class Foo:
    def foo(self, x):
        return x + 1
    
    def bar(self, x):
        if x >= 10:  # Changed comparison
            return x + 2.0  # Added decimal
        else:
            return x + 3

